In [15]:
import yaml
from yaml import SafeLoader

class Event:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

class MomentTensor:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

def event_constructor(loader, node):
    return Event(**loader.construct_mapping(node))

def moment_tensor_constructor(loader, node):
    return MomentTensor(**loader.construct_mapping(node))

class CustomLoader(SafeLoader):
    pass

CustomLoader.add_constructor('!pf.Event', event_constructor)
CustomLoader.add_constructor('!pf.MomentTensor', moment_tensor_constructor)

#with open('../../report/flegrei_2018_09_18_21_36_41/cmt_devi_near_05_2_far_005_01_td_fd_flegrei_2018_09_18_21_36_41/event.solution.mean.yaml', 'r') as file:
#    data = yaml.load(file, Loader=CustomLoader)


In [16]:
import os
import pandas as pd
import yaml
from pyrocko import model

# Step 1: Specificare la cartella da cui leggere i file
dir_reports = '/Users/giaco/UNI/PhD_CODE/GIT/CAMPI_FLEGREI_moment_tensor/report'

# Step 2: Creare il DataFrame vuoto con le colonne predefinite
df = pd.DataFrame(columns=['event_name','latitude','longitude','strike','dip','rake','magnitude','depth'])

# Step 3: Iterare sui file e aggiungere i dati riga per riga
for ev_name in os.listdir(dir_reports):
    if ev_name.startswith('flegrei'):
        dir_ev = os.path.join(dir_reports, ev_name)

        # take lat and long from catalogue
        cat_name='../../CAT/catalogue_flegrei_mag_2_5.pf'   
        cat_pf=model.load_events(cat_name)
        for ev in cat_pf:
            if ev.name==ev_name:
                lat=float(ev.lat)
                long=float(ev.lon)

        for config_name in os.listdir(dir_ev):
            config_name_str='cmt_devi_near_05_2_far_008_03_td_fd_flegrei' #CHANGE CONFIG NAME
            if config_name.startswith(config_name_str):       
                dir_config= os.path.join(dir_ev,config_name)
                event_solution_name=os.path.join(dir_config,'event.solution.best.yaml')     #USING BEST SOLUTIONS
                with open(event_solution_name, 'r') as file:
                    data = yaml.load(file, Loader=CustomLoader)
                depth=float(data.depth)
                moment_tensor = data.moment_tensor
                strike=float(moment_tensor.strike1)
                dip=float(moment_tensor.dip1)
                rake=float(moment_tensor.rake1)
                magnitude=float(moment_tensor.magnitude)

                # Step 4: Creare un dizionario con i valori della riga
                row_data = {'longitude':long,'latitude':lat,'strike':strike,
                            'dip':dip,'rake':rake,'magnitude':magnitude,
                            'depth':depth,'event_name':ev_name,}
                # Step 5: Aggiungere la riga al DataFrame
                if magnitude >= 1.:     #add only events of magnitude >= 1
                    df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)

# Step 6: Salvare il DataFrame come file CSV
df.to_csv('focal_mechanism_'+config_name_str+'.csv', index=False)


/var/folders/_5/hfq6n3xn3n53jvjz81t_f2r80000gn/T/ipykernel_12642/3762656139.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)
